Questions...
- Compare with non-rehab birds (all species)
    - How many rehab versus non-rehab birds are there? 
    - How many detections for each?
        - Detected rehabs: How many were detected elsewhere (repeat tags)?
- Movement 
    - How many birds are contained within a particular area?
    - Map the detections by color (rehab versus not)
        - Receivers:
            - Which receivers picked up birds?
            - Where are those receivers located?

Need to determine which tags were picked up twice (to compare movement trends)

## Dependencies

In [2]:
!pip install geopandas

from matplotlib import pyplot as plt
from math import sin, cos, sqrt, atan2, radians
import numpy as np
import pandas as pd
import geopandas as gpd
from datetime import datetime
from shapely.geometry import Point, MultiPoint
from shapely.ops import nearest_points

     |████████████████████████████████| 901kB 5.1MB/s 
     |████████████████████████████████| 11.2MB 42.8MB/s 
     |████████████████████████████████| 17.9MB 24.7MB/s 
  Created wheel for munch: filename=munch-2.3.2-py2.py3-none-any.whl size=6613 sha256=351bed14ec89c9b3eb28549b940605d163540e2849c94198e2cde51fdd0a883b
  Stored in directory: /root/.cache/pip/wheels/db/bf/bc/06a3e1bfe0ab27d2e720ceb3cff3159398d92644c0cec2c125
Successfully built munch


In [3]:
# 1.2 import for Google Drive - mount 
# Mount Google Drive to import the spreadsheet
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


In [4]:
# 1.3 import for Google Drive - dependencies
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

#https://drive.google.com/open?id=1uHkWewXdOeLrIcsFBzT9-v7zezAWmT6-

     |████████████████████████████████| 993kB 4.9MB/s 


In [5]:
# 1.3 import for Google Drive -import
# Download a file based on its file ID.
#
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
file_id = '1lHrbCx-PUdARIKWE8e37ebqHDGGzsYdo' # nanotag csv
file_id2 = '1kujAosnNw-Q-8reuZcjrbbrmcHtBx3V6' # detections copy csv

downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('2018 Collision nanotag data.csv') 

downloaded2 = drive.CreateFile({'id': file_id2})
downloaded2.GetContentFile('copy_detction_data.csv') 
# print('Downloaded content "{}"'.format(downloaded.GetContentString()))

#Create dataframe
df_nanotags2 = pd.read_csv('2018 Collision nanotag data.csv', encoding = "mac_roman")
df_detections = pd.read_csv('copy_detction_data.csv', encoding = "mac_roman")




'''
nysdf = pd.read_csv("/Users/Monroe/Documents/Programming/Projects/nys-environmental-remediation-sites/environmental-remediation-sites.csv")

## Use this with an input to hide my filepath from online... 
## train_df = pd.read_csv(os.path.join(PATH,'train.csv')
'''

'\nnysdf = pd.read_csv("/Users/Monroe/Documents/Programming/Projects/nys-environmental-remediation-sites/environmental-remediation-sites.csv")\n\n## Use this with an input to hide my filepath from online... \n## train_df = pd.read_csv(os.path.join(PATH,\'train.csv\')\n'

## Cleaning Data

In [0]:
# 2. Clean data
# a. Remove rows with Null bird species
df_detections = df_detections.dropna(subset=['speciesEN.x'])

# b. Changing tag 28452 to Oven Bird - changing rehab, species, release site
df_detections.loc[df_detections['motusTagID'] == 28452, ['rehab', 'speciesEN.y', 'release.site']] = 'Y','OVEN', 'LENSC'

# c. on nanotags df, change cols to lower case to match tags
df_nanotags2 = df_nanotags2.rename(str.lower, axis='columns')

# d. join the release time and date cols to the original df
df_detections = pd.merge(df_detections,df_nanotags2[['tag', 
            'release date (utc)', 'release time (utc).1', 'time in custody (days)', 'latitude',
       'longitude']],on='tag', how='left')

df_detections = df_detections.rename(columns = {'release time (utc).1': 'releaseTimeUTC', 
                                                'release date (utc)': 'releaseDate', 
                                               'latitude' : 'relLat', 
                                               'longitude' : 'relLong'})

# e. rename release site names so that FRYS is uniform. only 4 different release sites
df_detections.loc[df_detections['release.site'] == 'Frys', ['release.site']] = 'FRYS'

df_detections.loc[df_detections['release.site'] == 'frys', ['release.site']] = 'FRYS'


In [0]:
# 3. Make datetime uniform...

# a. convert the day and abbrev month as new col
df_detections['date2'] = pd.to_datetime(df_detections['rehab.intake.date'], format='%d-%b')

# b.  add in year as 2018
df_detections['date2'] = df_detections['date2'].apply(lambda dt: dt.replace(year=2018, hour = 12, minute = 0, second = 0))

# c. replace with original col
df_detections['rehab.intake.date'] = df_detections['date2']
df_detections = df_detections.drop(columns=['date2'], axis=1)

# d. convert 'ts.h' as datetime
df_detections['ts.h'] = pd.to_datetime(df_detections['ts.h'], format='%Y-%m-%d %H:%M:%S')

In [8]:
#df_nanotags2['tag type'].unique()
df_detections['release.site'].unique()


array(['FRYS', 'LENSC', 'PGH', 'PARC'], dtype=object)

In [0]:
# 4. Days since rehab & remove bad dates

# a. Calculate number of days since rehab release and since first detection
# New col for days since rehab
df_detections['DaysSinceRehab'] = (df_detections['ts.h'] - df_detections['rehab.intake.date']).dt.days

# b. remove bad entries when detection date comes before rehab
df_detections = df_detections.drop(df_detections[df_detections.DaysSinceRehab <= 0].index)

# BELOW IS OLD - The average number of days since rehab and intake, selecting only for positive values

# PROBLEM: some of the days since rehab are negative... why?
# Get the unique tags when this occurs
# df_OffDates = df_detections.loc[df_detections['DaysSinceRehab'] <= 0]
# df_OffDates.motusTagID.unique()

## EDA for Species and Receivers

In [10]:
# 4. EDA for speices and receivers
# a. How many unique species? 6 total
print('Number of Different Species = ' + str(len(df_detections['speciesEN.x'].unique())))

# b. How many different receivers?
print('Number of Different Receivers = ' + str(len(df_detections['recvDeployName'].unique())))

# c. How many unique species? 6 total
print(df_detections['speciesEN.x'].unique())

# d. How many unqiue birds tagged & detected ? - 36 total
print('Total Unique Birds Tagged & Detected: ' + str(len(df_detections.motusTagID.unique())))

# d. Group receivers by tags and total number of hits
df_detections.groupby(['recvDeployName', 'motusTagID']).size().reset_index(name = 'count').sort_values(by = 'count', ascending = False)

Number of Different Species = 6
Number of Different Receivers = 37
['White-throated Sparrow' 'American Woodcock' 'Gray Catbird' 'Ovenbird'
 'Magnolia Warbler' 'Wood Thrush']
Total Unique Birds Tagged & Detected: 36


,recvDeployName,motusTagID,count
50,PARC Honey Hut,28436,11972
42,PARC Banding,28763,7310
41,PARC Banding,28445,1302
39,PARC Banding,28434,1256
44,PARC Honey Hut,28415,706
57,PARC Honey Hut,28457,679
46,PARC Honey Hut,28422,129
52,PARC Honey Hut,28446,110
53,PARC Honey Hut,28448,96
51,PARC Honey Hut,28441,89


In [11]:
# 5. EDA by Species - AMWO 
# Separate by species
df_woodcock = df_detections.loc[df_detections['speciesEN.x'] == "American Woodcock"]

df_AMWOrehab = df_woodcock.loc[df_woodcock['rehab'] == "Y"]
print('AMWO Rehab Count:' + str(len(df_AMWOrehab.motusTagID.unique()))) # of the rehab, total number unique banded
print('AMWO Rehab Tags: ' + str(df_AMWOrehab.motusTagID.unique()))# of the rehab, unique tags

df_AMWOwild = df_woodcock.loc[df_woodcock['rehab'] == "N"]
print('AMWO Wild Count: ' + str(len(df_AMWOwild.motusTagID.unique()))) # of the rehab, total number unique banded
print('AMWO Wild Tags: ' + str(df_AMWOwild.motusTagID.unique())) # of the rehab, unique tags

print('Total Detections Rehab: ' + str(len(df_woodcock.loc[df_woodcock['rehab'] == "Y"]))) #total detections rehab
print('Total Detections Wild: ' + str(len(df_woodcock.loc[df_woodcock['rehab'] == "N"]))) #total detections wild

# Release receivers that picked up tags - groupby size
df_woodcock.groupby('recvDeployName').size()

AMWO Rehab Count:4
AMWO Rehab Tags: [28451 28449 28420 28428]
AMWO Wild Count: 1
AMWO Wild Tags: [28445]
Total Detections Rehab: 112
Total Detections Wild: 1303


recvDeployName
39.8:-79.7                     1
Earl_Rowe_PP                   1
Girard                         3
Haldimand Raptor Reserve       5
Long_Point_Tip                 5
Nanticoke                      3
PARC Banding                1302
PARC Honey Hut                93
PresqueIsle                    1
Ruthven                        1
dtype: int64

In [12]:
# 6. EDA by Species - WTSP 
# Separate by species
df_wtsparrow = df_detections.loc[df_detections['speciesEN.x'] == "White-throated Sparrow"]

df_WTSPrehab = df_wtsparrow.loc[df_wtsparrow['rehab'] == "Y"]
print('WTSP Rehab Count:' + str(len(df_WTSPrehab.motusTagID.unique()))) # of the rehab, total number unique banded
print('WTSP Rehab Tags: ' + str(df_WTSPrehab.motusTagID.unique())) # of the rehab, unique tags

df_WTSPwild = df_wtsparrow.loc[df_wtsparrow['rehab'] == "N"]
print('WTSP Wild Count: ' + str(len(df_WTSPwild.motusTagID.unique()))) # of the rehab, total number unique banded
print('WTSP Wild Tags: ' + str(df_WTSPwild.motusTagID.unique())) # of the rehab, unique tags

print('Total Detections Rehab: ' + str(len(df_wtsparrow.loc[df_wtsparrow['rehab'] == "Y"]))) #total detections rehab
print('Total Detections Wild: ' + str(len(df_wtsparrow.loc[df_wtsparrow['rehab'] == "N"]))) #total detections wild

# Release receivers that picked up tags - groupby size
df_wtsparrow.groupby('recvDeployName').size()

WTSP Rehab Count:1
WTSP Rehab Tags: [28423]
WTSP Wild Count: 4
WTSP Wild Tags: [28414 28416 28426 28454]
Total Detections Rehab: 6
Total Detections Wild: 7


recvDeployName
Binbrook               1
Brighton               1
Earl_Rowe_PP           1
Flamborough_Quarry     1
Gerdau                 2
Long_Point_Tip         6
Morgan Rd. Office 1    1
dtype: int64

In [13]:
# 7. EDA by Species - GRCA 
# Separate by species
df_gcatbird = df_detections.loc[df_detections['speciesEN.x'] == "Gray Catbird"]

df_GRCArehab = df_gcatbird.loc[df_gcatbird['rehab'] == "Y"]
print('GRCA Rehab Count: ' + str(len(df_GRCArehab.motusTagID.unique()))) # of the rehab, total number unique banded
print('GRCA Rehab Tags: ' + str(df_GRCArehab.motusTagID.unique())) # of the rehab, unique tags


df_GRCAwild = df_gcatbird.loc[df_gcatbird['rehab'] == "N"]
print('GRCA Wild Count: ' + str(len(df_GRCAwild.motusTagID.unique()))) # of the wild, total number unique banded
print('GRCA Wild Tags: ' + str(df_GRCAwild.motusTagID.unique())) # of the wild, unique tags

print('Total Detections Rehab: ' + str(len(df_gcatbird.loc[df_gcatbird['rehab'] == "Y"]))) #total detections rehab
print('Total Detections Wild: '+ str(len(df_gcatbird.loc[df_gcatbird['rehab'] == "N"]))) #total detections wild

# Release receivers that picked up tags - groupby size
df_gcatbird.groupby('recvDeployName').size()

GRCA Rehab Count: 6
GRCA Rehab Tags: [28446 28429 28441 28436 28453 28422]
GRCA Wild Count: 3
GRCA Wild Tags: [28460 28458 24376]
Total Detections Rehab: 12347
Total Detections Wild: 11


recvDeployName
43.6:-79.35              4
Binbrook                 1
Gerdau                   1
Girard                   2
Lakeview Landfill        3
Long_Point_Tip           1
PARC Banding             3
PARC CMNH                7
PARC Honey Hut       12326
PARC Mt. Laurel          1
PKR_salt_pannes          4
PresqueIsle              1
WAQT                     4
dtype: int64

In [14]:
# 8. EDA by Species - OVEN 
# Separate by species
df_ovenbird = df_detections.loc[df_detections['speciesEN.x'] == "Ovenbird"]

df_OVENrehab = df_ovenbird.loc[df_ovenbird['rehab'] == "Y"]
print('OVEN Rehab Count: ' + str(len(df_OVENrehab.motusTagID.unique()))) # of the rehab, total number unique banded
print('OVEN Rehab Tags: ' + str(df_OVENrehab.motusTagID.unique())) # of the rehab, unique tags


df_OVENwild = df_ovenbird.loc[df_ovenbird['rehab'] == "N"]
print('OVEN Wild Count: ' + str(len(df_OVENwild.motusTagID.unique()))) # of the wild, total number unique banded
print('OVEN Wild Tags: ' + str(df_OVENwild.motusTagID.unique())) # of the wild, unique tags

print('Total Detections Rehab: '+ str(len(df_ovenbird.loc[df_ovenbird['rehab'] == "Y"]))) #total detections rehab
print('Total Detections Wild: ' + str(len(df_ovenbird.loc[df_ovenbird['rehab'] == "N"]))) #total detections wild

# Release receivers that picked up tags - groupby size
df_ovenbird.groupby('recvDeployName').size()

OVEN Rehab Count: 4
OVEN Rehab Tags: [28425 28452 28427 24345]
OVEN Wild Count: 2
OVEN Wild Tags: [28761 28450]
Total Detections Rehab: 74
Total Detections Wild: 3


recvDeployName
43.6:-79.35                 6
Churchill - CNSC            2
Churchill - Fen            15
Curries                     1
D'Estimauville              8
Dunkirk Lighthouse          1
Merlin                      1
PARC Honey Hut             41
Rondeau_Provincial_Park     1
Sunset Farm                 1
dtype: int64

In [15]:
# 9. EDA by Species - WOTH 
# Separate by species
df_woodthrush = df_detections.loc[df_detections['speciesEN.x'] == "Wood Thrush"]
df_WOTHrehab = df_woodthrush.loc[df_woodthrush['rehab'] == "Y"]
df_WOTHwild = df_woodthrush.loc[df_woodthrush['rehab'] == "N"]
print('WOTH Rehab Count: ' + str(len(df_WOTHrehab.motusTagID.unique()))) # of the rehab, total number unique banded
print('WOTH Rehab Tags: ' + str(df_WOTHrehab.motusTagID.unique())) # of the rehab, unique tags

print('WOTH Wild Count: ' + str(len(df_WOTHwild.motusTagID.unique()))) # of the wild, total number unique banded
print('WOTH Wild Tags: ' + str(df_WOTHwild.motusTagID.unique())) # of the wild, unique tags

print('Total Detections Rehab: ' + str(len(df_woodthrush.loc[df_woodthrush['rehab'] == "Y"])))
print('Total Detections Wild: ' + str(len(df_woodthrush.loc[df_woodthrush['rehab'] == "N"])))

# Release receivers that picked up tags - groupby size
df_woodthrush.groupby('recvDeployName').size()

WOTH Rehab Count: 3
WOTH Rehab Tags: [28457 28448 28455]
WOTH Wild Count: 3
WOTH Wild Tags: [28763 28434 28440]
Total Detections Rehab: 801
Total Detections Wild: 8572


recvDeployName
43.6:-79.35              2
Dunkirk Lighthouse       3
Falconer_Farm            3
Girard                   3
Lakeview Landfill        4
PARC Banding          8572
PARC Honey Hut         782
Waterford_Quarry         1
the Wilds                3
dtype: int64

In [16]:
# 10. EDA by Species - MAWA 
# Separate by species
df_magwarbler = df_detections.loc[df_detections['speciesEN.x'] == "Magnolia Warbler"]

df_MWrehab = df_magwarbler.loc[df_magwarbler['rehab'] == "Y"]
print('MAWA Rehab Count: ' + str(len(df_MWrehab.motusTagID.unique()))) # of the rehab, total number unique banded
print('MAWA Rehab Tags: ' + str(df_MWrehab.motusTagID.unique())) # of the rehab, unique tags


df_MWwild = df_magwarbler.loc[df_magwarbler['rehab'] == "N"]
print('MAWA Wild Count: ' + str(len(df_MWwild.motusTagID.unique()))) # of the wild, total number unique banded
print('MAWA Wild Count: ' + str(df_MWwild.motusTagID.unique())) # of the wild, unique tags

print('Total Detections Rehab: ' + str(len(df_magwarbler.loc[df_magwarbler['rehab'] == "Y"])))
print('Total Detections Wild: ' + str(len(df_magwarbler.loc[df_magwarbler['rehab'] == "N"])))

# Release receivers that picked up tags - groupby size
df_magwarbler.groupby('recvDeployName').size()

MAWA Rehab Count: 3
MAWA Rehab Tags: [28433 28419 28415]
MAWA Wild Count: 2
MAWA Wild Count: [28412 28413]
Total Detections Rehab: 716
Total Detections Wild: 3


recvDeployName
43.6:-79.35               1
47.45:-70.05              1
D'Estimauville            1
Napanee LOSH              1
PARC Honey Hut          706
Pointe ‡ la Loupe         2
Pointe-aux-Outardes       1
St-Roch-des-Aulnaies      4
Sunset Farm               2
dtype: int64

In [17]:
# 11. Receivers EDA - Basic 
# a. num of different release sites
print('Total # of release sites: ' + str(len(df_detections['release.site'].unique()))) # different release sites
# b. num of different recvs
print('Total # of receivers: ' + str(len(df_detections['recvDeployName'].unique())))
# c. names of different recvs
print(df_detections['recvDeployName'].unique()) # different receiver deployments
# d. group recvs by their total count, top 5
df_detections.groupby(['recvDeployName']).size().reset_index(name = 'count').sort_values(by = 'count', ascending = False).head()

Total # of release sites: 4
Total # of receivers: 37
['Long_Point_Tip' 'Girard' 'PresqueIsle' 'Brighton' 'Gerdau'
 'PARC Honey Hut' 'Binbrook' 'Rondeau_Provincial_Park'
 'Flamborough_Quarry' 'Earl_Rowe_PP' 'Curries' '43.6:-79.35' 'Merlin'
 'Falconer_Farm' 'Waterford_Quarry' 'PARC CMNH' 'Lakeview Landfill'
 'PARC Banding' 'Dunkirk Lighthouse' 'PARC Mt. Laurel' 'Napanee LOSH'
 "D'Estimauville" 'Morgan Rd. Office 1' 'St-Roch-des-Aulnaies'
 '47.45:-70.05' 'Pointe ‡ la Loupe' 'Pointe-aux-Outardes'
 'PKR_salt_pannes' 'WAQT' 'Sunset Farm' 'Churchill - CNSC'
 'Churchill - Fen' 'the Wilds' 'Haldimand Raptor Reserve' 'Ruthven'
 'Nanticoke' '39.8:-79.7']


,recvDeployName,count
24,PARC Honey Hut,13948
22,PARC Banding,9877
6,Churchill - Fen,15
1,43.6:-79.35,13
17,Long_Point_Tip,12


In [18]:
# 12. Receivers EDA - See which recvs got which tags
# a. Make new receiver df 
df_recByTag = df_detections.groupby(['recvDeployName', 
                'motusTagID']).size().reset_index(name = 'count').sort_values(by = 'count', ascending = False)

# b. and then join on rehab col
df_recByTag = pd.merge(df_recByTag,df_detections[['motusTagID', 'rehab']],on='motusTagID', how='left')

# c. Removing duplicates caused by count
df_recByTag = df_recByTag.drop_duplicates()
df_recByTag.head(50)

# d. make new df for unique tags of each receiver
df_recUniTags = df_recByTag.groupby('recvDeployName', sort=False).motusTagID.unique().to_frame('UniqueTags')
df_recUniTags['TotalTags'] = df_recUniTags['UniqueTags'].str.len()

# e. recvs with more than one tag
df_recvslist = df_recUniTags.loc[df_recUniTags['TotalTags'] > 1].reset_index()

# f. as list - recvs with more than one tag
list_recvsover1 = df_recvslist['recvDeployName'].tolist()

print('Number of receivers that detected more than one tag: ' + str(len(list_recvsover1)))


# e. view top 5 receivers by the number of unique tags they detected
print(df_recUniTags.nlargest(5, 'TotalTags'))

Number of receivers that detected more than one tag: 13
                                                       UniqueTags  TotalTags
recvDeployName                                                              
PARC Honey Hut  [28436, 28415, 28457, 28422, 28446, 28448, 284...         14
PARC Banding                  [28763, 28445, 28434, 28440, 24376]          5
Long_Point_Tip                [28451, 28423, 28460, 28414, 28426]          5
43.6:-79.35                   [28452, 28458, 28455, 28761, 28412]          5
Girard                                      [28457, 28451, 28441]          3


In [19]:
# 13. Receivers EDA - Which tags were detected by more than one receiver?
# a. as new df - group by tagID and then put all the recv names in a list for each as count 
df_tagsByRecHits = df_recByTag.groupby('motusTagID',
                                        sort=False).recvDeployName.unique().to_frame('recvs').reset_index()
# b. view the total recvs as length of recvs list
df_tagsByRecHits['TotalRecvs'] = df_tagsByRecHits['recvs'].str.len()

# c. extract tags in the new df as a list, based on where totalRecvs is greater than X 
df_tagslist = df_tagsByRecHits.loc[df_tagsByRecHits['TotalRecvs'] > 1]
list_over1 = df_tagslist['motusTagID'].tolist()

df_tagslist3 = df_tagsByRecHits.loc[df_tagsByRecHits['TotalRecvs'] >= 3]
list_over3 = df_tagslist3['motusTagID'].tolist()

df_tagslist5 = df_tagsByRecHits.loc[df_tagsByRecHits['TotalRecvs'] >= 5]
list_over5 = df_tagslist5['motusTagID'].tolist()

#d. how many tags detected more ONCE, more than 3 times, more than 5 times? 
print('Number of Tags detected more than once: ' + str(len(list_over1)))
print('Number of Tags detected at least 3 times: ' + str(len(list_over3)))
print('Number of Tags detected at least 5 times: ' + str(len(list_over5)))



Number of Tags detected more than once: 16
Number of Tags detected at least 3 times: 9
Number of Tags detected at least 5 times: 3


In [20]:
# 13.1 Receivers EDA - View which recvs picked up rehab and wild birds
# a. grouping recvs, by unique rehab detections - putting them in a col called count 
df_recUniRehab = df_recByTag.groupby('recvDeployName', 
                                     sort=False).rehab.unique().to_frame(name = 'count').reset_index()
# b. from col count, getting length as new col 
df_recUniRehab['wildNrehab'] = df_recUniRehab['count'].str.len()
df_recUniRehab.head()

,recvDeployName,count,wildNrehab
0,PARC Honey Hut,[Y],1
1,PARC Banding,[N],1
2,Churchill - Fen,[Y],1
3,D'Estimauville,"[Y, N]",2
4,PARC CMNH,[Y],1


In [0]:
# 13.2 Receivers EDA - View which recvs picked up rehab and wild birds
# FINICKY 
# c. map true false using a dictionary
di = {1: False, 2: True}
df_recUniRehab['wildNrehab'] = df_recUniRehab['wildNrehab'].replace(di)

# d. save which recvs are true as new df
df_recUniRehab2 = df_recUniRehab.loc[df_recUniRehab['wildNrehab'] == True].reset_index()

# BELOW IS OLD 
# df_recUniRehab.head(10)
#df_recUniRehab.info()

# df_recUniRehab[df_recUniRehab['wildNrehab'== 'TRUE']]
#df_recUniRehab.loc[df_recUniRehab['wildNrehab'] == True]

In [22]:
# 13.3 Receivers EDA - View which recvs picked up rehab and wild birds
# e. using last df - put the recvs names with both detections as list
list_recvsWildnRehab = df_recUniRehab2['recvDeployName'].tolist()
# f. view list
print('Recvs detecting both wild and rehab: ' + str(list_recvsWildnRehab))

Recvs detecting both wild and rehab: ["D'Estimauville", 'Long_Point_Tip', '43.6:-79.35', 'Dunkirk Lighthouse']


## Probability of Detection

In [23]:
# 14. Review probability of being detected

# a. what is the total number of detections
df_rehabOnly = df_detections.loc[df_detections['rehab'] == 'Y']
print(df_rehabOnly.shape) # shows there are 14,056 observations

# b. how many unique recvs for rehab only? 
print('Total number rehab recvs: ' + str(len(df_rehabOnly['recvDeployName'].unique())))

# c. how many different birds are rehab? 
print('Total number rehab birds: ' + str(len(df_rehabOnly['motusTagID'].unique())))

# d. what is the total number of detections for just rehab birds
df_wildOnly = df_detections.loc[df_detections['rehab'] == 'N']
print(df_wildOnly.shape) # shows there are 9899 observations

# e. how many unique recvs for wild only? 
print('Total number wild recvs: ' +str(len(df_wildOnly['recvDeployName'].unique())))

# f. how many different birds are wild? 
print('Total number wild birds: ' + str(len(df_wildOnly['motusTagID'].unique())))

(14056, 23)
Total number rehab recvs: 29
Total number rehab birds: 21
(9899, 23)
Total number wild recvs: 12
Total number wild birds: 15


In [24]:
# NORMALIZED
# 14. Review probability of being detected

df_genNrml = df_detections.sort_values('ts.h').drop_duplicates('recvDeployName',keep='last')

# a. what is the total number of detections
df_rehabOnly2 = df_genNrml.loc[df_genNrml['rehab'] == 'Y']
print(df_rehabOnly2.shape) # shows there are 14,056 observations

# b. how many unique recvs for rehab only? 
print('Total number rehab recvs: ' + str(len(df_rehabOnly2['recvDeployName'].unique())))

# c. how many different birds are rehab? 
print('Total number rehab birds: ' + str(len(df_rehabOnly2['motusTagID'].unique())))

# d. what is the total number of detections for just rehab birds
df_wildOnly2 = df_genNrml.loc[df_detections['rehab'] == 'N']
print(df_wildOnly2.shape) # shows there are 9899 observations

# e. how many unique recvs for wild only? 
print('Total number wild recvs: ' +str(len(df_wildOnly2['recvDeployName'].unique())))

# f. how many different birds are wild? 
print('Total number wild birds: ' + str(len(df_wildOnly2['motusTagID'].unique())))

df_genNrml.head()


(29, 23)
Total number rehab recvs: 29
Total number rehab birds: 12
(8, 23)
Total number wild recvs: 8
Total number wild birds: 7


,Unnamed: 0,motusTagID,tagDeployLon,tagDeployLat,speciesEN.x,recvDeployName,ts.h,recvLat,recvLon,tag,rehab,rehab.intake.date,condition,age,sex,speciesEN.y,release.site,releaseDate,releaseTimeUTC,time in custody (days),relLat,relLong,DaysSinceRehab
5,6,28414,-80.0861,42.1602,White-throated Sparrow,Brighton,2018-05-02 04:40:51,44.00,-77.75,234,N,NaT,NaN,ASY`,M,WTSP,FRYS,5/1/18,8:20,NaN,42.160200,-80.086100,NaN
49,50,28425,-81.9377,41.4861,Ovenbird,Rondeau_Provincial_Park,2018-05-09 04:47:17,42.30,-81.85,229,Y,2018-05-06 12:00:00,mild,ASY,U,OVEN,LENSC,5/8/18,17:35,2.0,41.486113,-81.937744,2.0
51,52,28423,-81.9377,41.4861,White-throated Sparrow,Flamborough_Quarry,2018-05-09 05:00:00,43.30,-80.00,227,Y,2018-05-02 12:00:00,moderate,ASY,U,WTSP,LENSC,5/2/18,18:24,0.0,41.486113,-81.937744,6.0
74,75,28425,-81.9377,41.4861,Ovenbird,Curries,2018-05-09 08:09:25,43.05,-80.75,229,Y,2018-05-06 12:00:00,mild,ASY,U,OVEN,LENSC,5/8/18,17:35,2.0,41.486113,-81.937744,2.0
795,728,28427,-81.9377,41.4861,Ovenbird,Merlin,2018-05-15 03:52:53,42.20,-82.15,230,Y,2018-05-05 12:00:00,mild,ASY,U,OVEN,LENSC,5/8/18,17:24,3.0,41.486113,-81.937744,9.0


In [25]:
# 14.1 Review probability of being detected by species
df_probSpe = df_detections.groupby(['speciesEN.x', 'rehab']).size().reset_index()

df_probSpe

,speciesEN.x,rehab,0
0,American Woodcock,N,1303
1,American Woodcock,Y,112
2,Gray Catbird,N,11
3,Gray Catbird,Y,12347
4,Magnolia Warbler,N,3
5,Magnolia Warbler,Y,716
6,Ovenbird,N,3
7,Ovenbird,Y,74
8,White-throated Sparrow,N,7
9,White-throated Sparrow,Y,6


## Assessing Length of Stay Post Release

In [0]:
# 15. Find out which tower is nearest to the release sites

# a. zip release sites lat long in new coordinates col
df_detections['relCoordinates'] = list(zip(df_detections['relLong'], df_detections['relLat'])) 

list_relCoords = df_detections['relCoordinates'].tolist()

list_relCoords = list( dict.fromkeys(list_relCoords)) # make list as dict to remove duplicates




In [0]:
# 15.1 Find out which tower is nearest to the release sites

# b. zip recv lat long in new coordinates col
df_detections['recvsCoordinates'] = list(zip(df_detections['recvLon'], df_detections['recvLat'])) 

list_recvCoords = df_detections['recvsCoordinates'].tolist()

list_recvCoords = list( dict.fromkeys(list_recvCoords)) # make list as dict to remove duplicates

In [28]:
# 15.2 Find out which tower is nearest to the release sites

# c. make both coord lists multi Point
list_relCoords = MultiPoint(list_relCoords)
coord0_frys = list_relCoords[0]
coord1_lensc = list_relCoords[1]
coord2_pgh = list_relCoords[2]
coord3_parc = list_relCoords[3]
list_recvCoords = MultiPoint(list_recvCoords)

print(len(list_relCoords))
print(list_relCoords)
print(coord0_frys)
print(coord1_lensc)
print(coord2_pgh)
print(coord3_parc)
print(len(list_recvCoords))
# print(len(df_detections['recvDeployName'].unique()))

4
MULTIPOINT (-80.0861 42.1602, -81.937744 41.48611289999999, -79.9458113 40.4385688, -79.26740644 40.16353282)
POINT (-80.0861 42.1602)
POINT (-81.937744 41.48611289999999)
POINT (-79.9458113 40.4385688)
POINT (-79.26740644 40.16353282)
37


In [29]:
# 15.3 Find out which tower is nearest to the release sites

# d. use nearest points. relcoords as origin and recvs as destinations
near_frys = nearest_points(coord0_frys, list_recvCoords)
near_lensc = nearest_points(coord1_lensc, list_recvCoords)
near_pgh = nearest_points(coord2_pgh, list_recvCoords)
near_parc = nearest_points(coord3_parc, list_recvCoords)

# e. print out the nearest sites
# matched by reviewing the original df in excel
print(near_frys[1]) # PresqueIsle
print(near_lensc[1]) # PARC Honey Hut
print(near_pgh[1]) # PARC CMNH
print(near_parc[1]) # PARC Banding


POINT (-80.15000000000001 42.1)
POINT (-81.95 41.5)
POINT (-79.95 40.45)
POINT (-79.25 40.15)


In [0]:
# 15.4 Map nearest towers in a separate column

# f. add new column for nearest sites
#df_detections['nearestRecv']
#df_detections.loc[df_detections['release.site'] == 'FRYS', ['nearest.recv']] = 'PresqueIsle'

equiv = {'FRYS':'PresqueIsle', 'LENSC':'PARC Honey Hut', 'PGH':'PARC CMNH', 'PARC':'PARC Banding'}

df_detections['nearestRecv'] = df_detections['release.site'].map(equiv)

In [0]:
# 15.5 How many days do wild and rehab birds stay at their release site before departing

# extract out the very first detections for each tag, then see which ones match first detections to the nearest tower

# a. group the df by detections time in new df
# df_bytime = df_detections.groupby(pd.Grouper(key = 'ts.h', freq = 'M'))
df_bytime = df_detections.sort_values(by=['ts.h'])

In [0]:
# 15.5 Which birds were first detected by the towers cloest to their release sites?

# a. sort by time then extract out by the first tag IDs
# copied this from exmaining days since release, step 16
df_firsthits = df_detections.sort_values(by=['ts.h'])
df_firsthits.drop_duplicates(subset ='motusTagID', keep = 'first', inplace = True)

# b. match where nearestRecv == recv name
df_matchnear = df_firsthits.loc[df_firsthits['nearestRecv'] == df_firsthits['recvDeployName']].reset_index()

In [33]:
# c. print shape and unique tags of new matching df 
print(df_matchnear.shape) #shows 20 different obs
print(len(df_matchnear['motusTagID'].unique())) 

# d. which birds were detected by the nearest receiver? 
list_matchnear = df_matchnear['motusTagID'].tolist() # make a list of tags
print(list_matchnear)

# e. how many wild vs rehab?
print('Rehab Matching Near: ' + str(len(df_matchnear.loc[df_matchnear['rehab'] == 'Y'])))
print('Wild Matching Near: ' + str(len(df_matchnear.loc[df_matchnear['rehab'] == 'N'])))


(20, 27)
20
[28446, 28425, 28429, 28449, 28441, 28457, 28448, 28455, 28436, 28452, 28453, 28422, 28445, 24376, 28763, 28434, 28440, 28420, 28415, 28428]
Rehab Matching Near: 15
Wild Matching Near: 5


In [0]:
# 16. How many days do wild and rehab birds stay at their release site before departing

# extract out the very first detections for each tag, then see which ones match first detections to the nearest tower

# a. group the df by detections time in new df
# df_bytime = df_detections.groupby(pd.Grouper(key = 'ts.h', freq = 'M'))
df_bytime = df_detections.sort_values(by=['ts.h'])

In [0]:
# 16.1 How many days do wild and rehab birds stay at their release site before departing

# b. drop duplicates for first observations of unique tagID
df_bytime.drop_duplicates(subset ='motusTagID', keep = 'first', inplace = True) 

# c. combine the release time with dates as new col
df_bytime['relDateTime'] = pd.to_datetime(df_bytime['releaseDate'] + ' ' + df_bytime['releaseTimeUTC'])

# c. find the difference in days since release
df_bytime['DaysSinceRelease'] = (df_bytime['ts.h'] - df_bytime['relDateTime']).dt.days

# d. find the difference in mins since release
df_bytime['MinsSinceRelease'] = (df_bytime['ts.h'] - df_bytime['relDateTime']).astype('<m8[m]').astype(int)


In [0]:
# 16.2 How many days do wild and rehab birds stay at their release site before departing

# e. separate rehab from wild in new dfs
df_wildbytime = df_bytime.loc[df_bytime['rehab'] == 'N'].reset_index()
df_rehabbytime = df_bytime.loc[df_bytime['rehab'] == 'Y'].reset_index()

In [44]:
# 16.2 How many days do wild and rehab birds stay at their release site before departing

# f. calculate average number of days and mins since release for wild birds
print('Wild - Mean Days Since Release: ' + str(df_wildbytime['DaysSinceRelease'].mean()))

print('Wild - Mean Mins Since Release: ' + str(df_wildbytime['MinsSinceRelease'].mean()))

# g. calculate average number of days and mins since release for rehab birds
print('Rehab - Mean Days Since Release: ' + str(df_rehabbytime['DaysSinceRelease'].mean()))

print('Rehab - Mean Mins Since Release: ' + str(df_rehabbytime['MinsSinceRelease'].mean()))

df_timeSpe = df_bytime.groupby(['speciesEN.x', 'rehab'])['DaysSinceRelease'].mean().reset_index()

df_timeSpe


Wild - Mean Days Since Release: 6.266666666666667
Wild - Mean Mins Since Release: 9914.466666666667
Rehab - Mean Days Since Release: 7.0476190476190474
Rehab - Mean Mins Since Release: 10675.619047619048


,speciesEN.x,rehab,DaysSinceRelease
0,American Woodcock,N,22.000000
1,American Woodcock,Y,0.500000
2,Gray Catbird,N,4.666667
3,Gray Catbird,Y,0.000000
4,Magnolia Warbler,N,4.000000
5,Magnolia Warbler,Y,18.333333
6,Ovenbird,N,2.500000
7,Ovenbird,Y,20.250000
8,White-throated Sparrow,N,8.250000
9,White-throated Sparrow,Y,6.000000


## Speed and Distance

In [0]:
# 17. Is there a difference in migratory speed after departure 
# between wild and rehab birds (i.e. distance/time)?

# a. get the tags that were detected by more than one recv as new df
df_over1 = df_detections.loc[df_detections['motusTagID'].isin(list_over1)]

# b. sort it by ID and time
df_over1 = df_over1.sort_values(by=['motusTagID', 'ts.h'])

# c. clean cols
df_over1 = df_over1[['motusTagID','recvDeployName', 'ts.h', 'recvLat', 'recvLon', 'rehab', 'speciesEN.x']]

# d. calculate the difference between each timestamp
df_over1['timeDiff'] = df_over1['ts.h'].diff()

In [0]:
# 17.1 Is there a difference in migratory speed after departure 
# between wild and rehab birds (i.e. distance/time)?

# d. Haversine distance function
# vectorized haversine function
def haversine(lat1, lon1, lat2, lon2, to_radians=True, earth_radius=6371):
    """
    slightly modified version: of http://stackoverflow.com/a/29546836/2901002

    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees or in radians)

    All (lat, lon) coordinates must have numeric dtypes and be of equal length.

    """
    if to_radians:
        lat1, lon1, lat2, lon2 = np.radians([lat1, lon1, lat2, lon2])

    a = np.sin((lat2-lat1)/2.0)**2 + \
        np.cos(lat1) * np.cos(lat2) * np.sin((lon2-lon1)/2.0)**2

    return earth_radius * 2 * np.arcsin(np.sqrt(a))

# e. velocity function 
def calc_velocity(dist_mi, time_start, time_end):
    """Return 0 if time_start == time_end, avoid dividing by 0"""
    return dist_mi / (time_end - time_start).seconds if time_end > time_start else 0

In [0]:
# 17.2 Is there a difference in migratory speed after departure 
# between wild and rehab birds (i.e. distance/time)?

# g. create a new column for distance and calculate using shift and concat
df_over1['dist_mi'] = \
np.concatenate(df_over1.\
        groupby('motusTagID').apply(lambda x: haversine(x['recvLat'],
                        x['recvLon'],x['recvLat'].shift(), x['recvLon'].shift())).values)


# h. create a new column for velocity - mph
df_over1['velocity_mph'] = df_over1.apply(
    lambda row: calc_velocity(
        dist_mi=row['dist_mi'],
        time_start=row['ts.h'],
        time_end=row['ts.h'] + row['timeDiff']
    ), 
    axis = 1
)

In [0]:
# i. Mask over the first entries for each tag for the time difference
# the time difference isn't being calculated for each unique ID
# but rather each previous row - so this corrects the df

mask = df_over1.motusTagID != df_over1.motusTagID.shift()
df_over1.loc[mask, ['timeDiff']] = np.nan #this mask is dependent on the order of the rows


In [49]:
# 17.3 Is there a difference in migratory speed after departure 
# between wild and rehab birds (i.e. distance/time)?

# a. calculate mean distance traveled by tag
df_meanDisTag = df_over1.groupby('motusTagID')['dist_mi'].mean().reset_index().\
sort_values(by = 'dist_mi',ascending = False)

df_meanDisTag.head() # this df shows the mean distance traveled for each tag

# b. mean distance for rehab and wild
df_meanDisRehab = df_over1.groupby(['rehab'])['dist_mi'].mean().reset_index().\
sort_values(by = 'dist_mi',ascending = False)

df_meanDisRehab.head() # shows rehab birds flew farther

,rehab,dist_mi
1,Y,6.886218
0,N,0.766533


In [50]:
# 17.3 Is there a difference in migratory speed after departure 
# between wild and rehab birds (i.e. distance/time)?

# c. mean velocity by tag
df_meanSpdByTag = df_over1.groupby('motusTagID')['velocity_mph'].mean().reset_index().\
sort_values(by = 'velocity_mph',ascending = False)

# d. mean velocity for rehab and wild
df_meanSpd = df_over1.groupby(['rehab'])['velocity_mph'].mean().reset_index().\
sort_values(by = 'velocity_mph',ascending = False)

df_meanSpd.head() #the rehab birds flew faster

,rehab,velocity_mph
1,Y,0.003325
0,N,0.000167


In [51]:
# 17.3.1 We should normalize the data so that we just have duplicates for a different hit at a different date

# a. sort by the dates, then drop duplicates by recv - keeping the last
# this normalizes a little to reduce the repeat detections at the same receiver location, keeping only the final one. 
df_over1.columns
df_over1.head()

df_nrml = df_over1.sort_values('ts.h').drop_duplicates('recvDeployName',keep='last')
df_nrml.head()

# b. recalculate the mean speed by tag
df_meanSpdByTagN = df_nrml.groupby('motusTagID')['velocity_mph'].mean().reset_index().\
sort_values(by = 'velocity_mph',ascending = False)

# c. recalculate mean velocity for rehab and wild
df_meanSpdN = df_nrml.groupby(['rehab'])['velocity_mph'].mean().reset_index().\
sort_values(by = 'velocity_mph',ascending = False)

df_meanSpdN.head() 

,rehab,velocity_mph
1,Y,0.049876
0,N,0.011233


In [52]:
# 17.3.1 Normalization of the mean distance traveled

# a. recalculate calculate mean distance traveled by tag
df_meanDisTagN = df_nrml.groupby('motusTagID')['dist_mi'].mean().reset_index().\
sort_values(by = 'dist_mi',ascending = False)

df_meanDisTagN.head() # this df shows the mean distance traveled for each tag

,motusTagID,dist_mi
3,28414,246.289729
10,28452,158.008072
5,28425,105.991412
4,28423,83.495499
14,28460,63.294861


In [53]:
# b. recalculate mean distance for rehab and wild
df_meanDisRehabN = df_nrml.groupby(['rehab'])['dist_mi'].mean().reset_index().\
sort_values(by = 'dist_mi',ascending = False)

df_meanDisRehabN.head() # shows rehab birds flew farther

,rehab,dist_mi
0,N,69.997293
1,Y,41.913541


In [54]:
df_over1.loc[df_over1['speciesEN.x'] == "White-throated Sparrow"]

,motusTagID,recvDeployName,ts.h,recvLat,recvLon,rehab,speciesEN.x,timeDiff,dist_mi,velocity_mph
0,28414,Long_Point_Tip,2018-05-02 01:53:09,42.55,-80.05,N,White-throated Sparrow,NaT,NaN,0.000000
5,28414,Brighton,2018-05-02 04:40:51,44.00,-77.75,N,White-throated Sparrow,02:47:42,246.289729,0.024477
33,28423,Long_Point_Tip,2018-05-09 02:52:14,42.55,-80.05,Y,White-throated Sparrow,NaT,NaN,NaN
34,28423,Long_Point_Tip,2018-05-09 02:52:18,42.55,-80.05,Y,White-throated Sparrow,00:00:04,0.000000,0.000000
35,28423,Long_Point_Tip,2018-05-09 02:52:19,42.55,-80.05,Y,White-throated Sparrow,00:00:01,0.000000,0.000000
36,28423,Long_Point_Tip,2018-05-09 02:57:04,42.55,-80.05,Y,White-throated Sparrow,00:04:45,0.000000,0.000000
51,28423,Flamborough_Quarry,2018-05-09 05:00:00,43.30,-80.00,Y,White-throated Sparrow,02:02:56,83.495499,0.011320
61,28423,Earl_Rowe_PP,2018-05-09 06:37:17,44.15,-79.90,Y,White-throated Sparrow,01:37:17,94.856644,0.016251
41,28426,Long_Point_Tip,2018-05-09 03:21:18,42.55,-80.05,N,White-throated Sparrow,NaT,NaN,0.000000
48,28426,Binbrook,2018-05-09 04:40:23,43.10,-79.85,N,White-throated Sparrow,01:19:05,63.294861,0.013339


In [55]:
# c. Review by species
df_meanDisSpe = df_over1.groupby(['speciesEN.x','rehab'])['dist_mi'].aggregate([min, np.median, np.mean, max]).reset_index()
#sort_values(by = 'dist_mi',ascending = False)
df_meanDisSpe #the wild MagWarb was only detected once after take off - no mean available


,speciesEN.x,rehab,min,median,mean,max
0,American Woodcock,N,0.000000,0.000000,0.041962,54.634797
1,American Woodcock,Y,0.000000,15.513994,34.210675,228.850422
2,Gray Catbird,N,0.000000,0.000000,14.882404,63.294861
3,Gray Catbird,Y,0.000000,0.000000,10.567614,759.338736
4,Magnolia Warbler,N,523.179805,523.179805,523.179805,523.179805
5,Magnolia Warbler,Y,0.000000,0.000000,33.288491,120.923484
6,Ovenbird,Y,0.000000,0.000000,45.786651,1909.649067
7,White-throated Sparrow,N,63.294861,154.792295,154.792295,246.289729
8,White-throated Sparrow,Y,0.000000,0.000000,35.670429,94.856644
9,Wood Thrush,Y,0.000000,0.000000,1.311382,356.583344


In [56]:
# c. Review by species
df_meanDisSpe2 = df_nrml.groupby(['speciesEN.x','rehab'])['dist_mi'].aggregate([min, np.median, np.mean, max]).reset_index()
#sort_values(by = 'dist_mi',ascending = False)
df_meanDisSpe2 #the wild MagWarb was only detected once after take off - no mean available


,speciesEN.x,rehab,min,median,mean,max
0,American Woodcock,N,0.000000,27.317398,27.317398,54.634797
1,American Woodcock,Y,0.000000,11.119493,56.154276,228.850422
2,Gray Catbird,N,10.153591,45.610777,39.686410,63.294861
3,Gray Catbird,Y,0.000000,0.000000,3.981699,19.908495
4,Magnolia Warbler,N,NaN,NaN,NaN,NaN
5,Magnolia Warbler,Y,0.000000,9.150124,34.805933,120.923484
6,Ovenbird,Y,0.000000,50.414308,89.914580,316.016144
7,White-throated Sparrow,N,246.289729,246.289729,246.289729,246.289729
8,White-throated Sparrow,Y,83.495499,83.495499,83.495499,83.495499
9,Wood Thrush,Y,0.000000,0.000000,5.373095,26.865474


In [57]:
df_meanVeloSpe2 = df_nrml.groupby(['speciesEN.x','rehab'])['velocity_mph'].aggregate\
([min, np.median, np.mean, max]).reset_index()
df_meanVeloSpe2

,speciesEN.x,rehab,min,median,mean,max
0,American Woodcock,N,0.000000,0.000826,0.000826,0.001652
1,American Woodcock,Y,0.000000,0.002770,0.227406,0.794249
2,Gray Catbird,N,0.007935,0.010133,0.013756,0.023200
3,Gray Catbird,Y,0.000000,0.000000,0.004630,0.023149
4,Magnolia Warbler,N,NaN,NaN,NaN,NaN
5,Magnolia Warbler,Y,0.000000,0.010155,0.020430,0.061410
6,Ovenbird,Y,0.000000,0.000183,0.011089,0.057604
7,White-throated Sparrow,N,0.024477,0.024477,0.024477,0.024477
8,White-throated Sparrow,Y,0.011320,0.011320,0.011320,0.011320
9,Wood Thrush,Y,0.000000,0.000000,0.003161,0.015803
